Grundlage:
* https://js.langchain.com/v0.1/docs/modules/chains/popular/sqlite/
* https://js.langchain.com/v0.1/docs/integrations/toolkits/sql/

* https://python.langchain.com/docs/tutorials/sql_qa/

# Setup

In [25]:
import sqlite3
import langchain
from langchain_community.utilities import SQLDatabase

In [26]:
db_path = r"./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [27]:
print(db.dialect)
db.get_usable_table_names()

sqlite


['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [28]:
db.run("SELECT * FROM Artist LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

**Prompt Template** - Definieren, wie die Frage in eine SQL-Anweisung übersetzt wird.

In [45]:
sql_template = """
Du bist ein SQL-Experte, der ausschließlich lesende (SELECT-)Abfragen schreibt.
Deine Aufgabe ist es, aus einer natürlichen Spracheingabe eine korrekte SQL-Abfrage zu erstellen,
die exakt auf das folgende Datenbankschema abgestimmt ist. Beachte bitte:
  - Verwende nur die Tabellen und Spalten, die im Schema aufgeführt sind.
  - Schreibe ausschließlich SELECT-Abfragen, keine INSERT-, UPDATE- oder DELETE-Anweisungen.
  - Wenn du die Antwort nicht sicher ableiten kannst, gib eine höfliche Meldung zurück, dass die Frage unklar ist.

Bitte verwende exakt das folgende Format:

Frage: <die ursprüngliche Frage>
SQL-Abfrage: <die generierte SQL-Abfrage>
SQL-Ergebnis: <das Ergebnis der SQL-Abfrage>
Antwort: <eine klare, kurze und präzise Antwort auf die ursprüngliche Frage>

Nutze das unten stehende Datenbankschema:
{dbschema}

Frage: {question}
SQL-Abfrage:
"""


**Integrate the LLM**

In [46]:
import requests
from typing import List, Optional, Mapping, Any
from langchain.llms.base import LLM
from pydantic import BaseModel

class OllamaLLM(LLM):
    model: str
    base_url: str = "http://localhost:11434"
    token: Optional[str] = None
    temperature: float = 0.7
    max_tokens: int = 256

    @property
    def _llm_type(self) -> str:
        return "ollama"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs: Any) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        payload = {
            "model": self.model,
            "prompt": prompt,
            "options": {
                "temperature": self.temperature,
                "num_predict": self.max_tokens,
            },
            "token": self.token or ""
        }
        url = f"{self.base_url}/api/generate"
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        # Adjust this key depending on your Ollama API response format
        return data.get("response", "").strip()

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": self.model, "base_url": self.base_url}




In [41]:
!ollama list

NAME                     	ID          	SIZE  	MODIFIED     
qwen2.5-coder:7b         	2b0496514337	4.7 GB	31 hours ago	
deepseek-r1:8b           	28f8fd6cdc67	4.9 GB	3 days ago  	
llama3.2:1b-instruct-q4_0	53f2745c8077	770 MB	3 months ago	
llama3.2:1b              	baf6a787fdff	1.3 GB	3 months ago	
llama3.1:8b              	42182419e950	4.7 GB	4 months ago	
mistral:instruct         	f974a74358d6	4.1 GB	4 months ago	


In [47]:
import requests
import json
from langchain import PromptTemplate

def generate_sql(prompt, model="llama3.1:8B", token="your_api_token_here", temperature=0.7, max_tokens=256, base_url="http://localhost:11434"):
    # Prepare the payload with streaming disabled
    payload = {
        "model": model,
        "prompt": prompt,
        "options": {
            "temperature": temperature,
            "num_predict": max_tokens
        },
        "token": token,
        "stream": False  # Disable streaming to get a complete response
    }
    url = f"{base_url}/api/generate"
    response = requests.post(url, json=payload)
    response.raise_for_status()
    
    # Parse the complete JSON response
    data = response.json()
    # Assuming the API returns the generated SQL under the "response" key:
    return data.get("response", "").strip()

if __name__ == "__main__":
    # Define the SQL prompt template using LangChain's PromptTemplate
    sql_template = """
Du bist ein SQL-Experte, der ausschließlich SELECT-Abfragen generiert.
Deine Aufgabe ist es, aus der natürlichen Spracheingabe eine korrekte SQL-Abfrage zu erstellen,
die exakt auf das folgende Datenbankschema abgestimmt ist.
Verwende nur die aufgeführten Tabellen und Spalten.
Falls die Frage unklar ist, gib eine höfliche Meldung zurück.

Datenbankschema:
{dbschema}

Frage: {question}
SQL-Abfrage:
    """
    
    prompt_template = PromptTemplate(
        input_variables=["dbschema", "question"],
        template=sql_template
    )
    
    # Example database schema and question
    dbschema = "Album(AlbumID, Title, ArtistID); Artist(ArtistID, Name); Track(TrackID, Name, AlbumID)"
    question = "Zeige alle Tracks aus dem Album Jazz"
    
    # Generate the final prompt using the template
    prompt = prompt_template.format(dbschema=dbschema, question=question)
    sql_query = generate_sql(prompt)
    print("Generated SQL Query:")
    print(sql_query)


Generated SQL Query:
Um auf deine Frage zu antworten:

"Zeige alle Tracks aus dem Album Jazz"

Die korrekte SQL-Abfrage wäre:

```sql
SELECT T1.Name 
FROM Track AS T1 
INNER JOIN Album AS T2 ON T1.AlbumID = T2.AlbumID 
WHERE T2.Title = 'Jazz';
```

Dabei wird das Ergebnis basierend auf der Auflistung aller Tracks (T1) mit dem Album "Jazz" (T2) ausgewählt.
